# Predicting Flight Delay

Problem Set-up:
We define a delayed flight to be one that is delayed by >= 15 minutes. 
The prediction problem is to train a model that can classify flights, to predict if they will or will not be delayed.

Use case:
- The idea is that this model would be useful to choosing airlines, flightpaths, airports, at the time of booking, relatively in advance of the scheduled departure (days, weeks, months ahead of time). Therefore, the prediction problem will focus on features that can be known in advance, rather than predicting using day-off features like weather and previous flights from that day. 

Notes:
- We restrict the analysis to relatively large airport, those with more than 20 (domestic) flights a day

# This notebook: more features
* one-hot day of week
* one-hot month
* one-hot airline
* one-hot airport
* one-hot departure hour

In [1]:
# Imports
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd

import seaborn as sns
sns.set_style('white')

from sklearn.linear_model import LogisticRegression

In [2]:
# Import custom code
from flightdelay.fld import io as flio

# Load data

In [3]:
airlines_df, airports_df, flights_df = flio.load_data()

In [4]:
# Filter data to keys of interest
keys = ['MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'ORIGIN_AIRPORT',
       'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME',
       'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME',
       'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN',
       'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY']
flights_df = flights_df[keys]

In [5]:
flights_df = flights_df.dropna()

In [6]:
flights_df.head()

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,...,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY
0,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,...,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0
1,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,...,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0
2,1,1,4,US,840,N171US,SFO,CLT,20,18.0,...,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0
3,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,...,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0
4,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,...,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0


In [7]:
N_flights = 1000000
N_train = int(N_flights/2)
np.random.seed(0)
flight_take_idx = np.random.permutation(flights_df.index)[:N_flights]
df_trainval = flights_df.loc[flight_take_idx]

In [8]:
df_trainval.head(20)

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,...,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY
2167050,5,18,1,OO,2898,N866AS,LAX,SMF,1030,1027.0,...,1045.0,86.0,85.0,63.0,373,1148.0,4.0,1156,1152.0,-4.0
1395129,3,31,2,DL,228,N945DL,MCO,CVG,1321,1315.0,...,1325.0,129.0,133.0,119.0,757,1524.0,4.0,1530,1528.0,-2.0
2771766,6,23,2,DL,1812,N955DN,ATL,ORD,2205,2210.0,...,2229.0,123.0,114.0,89.0,606,2258.0,6.0,2308,2304.0,-4.0
5181045,11,20,5,EV,4869,N738EV,DTW,TVC,1200,1156.0,...,1208.0,67.0,58.0,43.0,207,1251.0,3.0,1307,1254.0,-13.0
947404,3,4,3,UA,283,N549UA,SFO,IAH,601,557.0,...,612.0,223.0,216.0,196.0,1635,1128.0,5.0,1144,1133.0,-11.0
3218377,7,20,1,WN,1119,N365SW,MKE,DAL,1800,1859.0,...,1909.0,145.0,144.0,129.0,850,2118.0,5.0,2025,2123.0,58.0
2666917,6,17,3,UA,1954,N16703,DCA,IAH,1744,2046.0,...,2113.0,200.0,198.0,161.0,1208,2254.0,10.0,2004,2304.0,180.0
3732761,8,20,4,WN,910,N703SW,SMF,LAX,1000,1011.0,...,1021.0,85.0,79.0,64.0,373,1125.0,5.0,1125,1130.0,5.0
3950971,9,2,3,DL,2065,N925AT,LAX,SEA,1910,1908.0,...,1923.0,162.0,152.0,128.0,954,2131.0,9.0,2152,2140.0,-12.0
4241971,9,21,1,WN,213,N791SW,DAL,DEN,1940,1944.0,...,1955.0,125.0,113.0,93.0,651,2028.0,9.0,2045,2037.0,-8.0


# Randomly take subset of data

# Compute features
* one-hot day of week
* one-hot month
* one-hot airline
* one-hot airport
* one-hot departure hour

### 0. Original feat

In [9]:
original_feat = {'f_distance':df_trainval['DISTANCE'].values}

In [10]:
y_delay = df_trainval['DEPARTURE_DELAY'].values

### 1. Day of week

In [11]:
def make_onehot_feat_dict(df, feat_key, feat_name):
    # Create features for each day of the week
    feat_vals = df[feat_key].values
    all_vals = np.unique(feat_vals)
    N_vals = len(all_vals)
    N_feat = N_vals - 1

    # Create keys
    keys = [0]*N_feat
    for i in range(N_feat):
        keys[i] = 'f_'+feat_name+'_'+ str(all_vals[i])

    # Create value for each training example in dict
    feat_dict = {}
    for i, k in enumerate(keys):
        this_day = all_vals[i]
        feat_dict[k] = feat_vals == this_day
    return feat_dict

In [12]:
daysfeat_dict = make_onehot_feat_dict(df_trainval, 'DAY_OF_WEEK', 'day')

### 2. Month

In [13]:
monthsfeat_dict = make_onehot_feat_dict(df_trainval, 'MONTH', 'month') 

### 3. Departing airport

In [14]:
dapfeat_dict = make_onehot_feat_dict(df_trainval, 'ORIGIN_AIRPORT', 'dap') 

### 4. Airline

In [15]:
alfeat_dict = make_onehot_feat_dict(df_trainval, 'AIRLINE', 'al') 

### 5. Departure hour

In [16]:
# Add departure hour as a feature
df_trainval['HOUR_DEPARTURE'] = np.floor(df_trainval['SCHEDULED_DEPARTURE'].values/100).astype(int)
df_trainval['HOUR_ARRIVAL'] = np.floor(df_trainval['SCHEDULED_ARRIVAL'].values/100).astype(int)
hrfeat_dict = make_onehot_feat_dict(df_trainval, 'HOUR_DEPARTURE', 'hr')

# Save new feature matrix

In [17]:
all_dicts = [original_feat, daysfeat_dict, monthsfeat_dict, dapfeat_dict, alfeat_dict, hrfeat_dict]
feat_dict = all_dicts[0].copy()
for d in all_dicts[1:]:
    feat_dict.update(d)

In [18]:
df_feat = pd.DataFrame.from_dict(feat_dict)
df_feat.head()

,f_al_AA,f_al_AS,f_al_B6,f_al_DL,f_al_EV,f_al_F9,f_al_HA,f_al_MQ,f_al_NK,f_al_OO,...,f_month_10,f_month_11,f_month_2,f_month_3,f_month_4,f_month_5,f_month_6,f_month_7,f_month_8,f_month_9
0,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,True,False,False,False,False
1,False,False,False,True,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
3,False,False,False,False,True,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False


# Prepare for training linear model

In [19]:
# zscore all features
from scipy.stats import zscore
df_feat = df_feat.apply(zscore)

In [20]:
# Declare DFs
X_train = df_feat.loc[:N_train-1].values
X_val = df_feat.loc[N_train:].values

In [21]:
# Determine labels
y_delay01 = y_delay > 15
y_train = y_delay01[:N_train]
y_val = y_delay01[N_train:]

# Train model

In [22]:
C = 100
model = LogisticRegression(C=C)
#model = LogisticRegression(C=C,fit_intercept=False)
#model = LogisticRegression(C=C,class_weight='balanced')
model.fit(X_train, y_train)

LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

# Evaluate model

In [23]:
preds_train = model.predict(X_train)
preds_val = model.predict(X_val)
print(model.score(X_train, y_train))
print(sum(model.predict(X_train)))
print(model.score(X_val, y_val))
print(sum(model.predict(X_val)))

0.8225
78
0.823364
61


In [24]:
from sklearn.metrics import confusion_matrix
cmat = confusion_matrix(y_train, preds_train)
print(cmat)

[[411204     32]
 [ 88718     46]]


In [25]:
print('True negative rate:', cmat[1,1]/sum(cmat[1]))
print('True positive rate:', cmat[0,0]/sum(cmat[0]))

True negative rate: 0.000518228110495
True positive rate: 0.999922185801


In [26]:
sum(y_train)

88764